In [112]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
from matplotlib.cm import ScalarMappable
from mpl_toolkits.axes_grid1.inset_locator import inset_axes

In [4]:
def get_label_rotation(angle, offset):
    # Rotation must be specified in degrees :(
    rotation = np.rad2deg(angle + offset)
    if angle <= np.pi:
        alignment = "right"
        rotation = rotation + 180
    else: 
        alignment = "left"
    return rotation, alignment

In [104]:
def add_labels(angles, values, labels, offset, ax):
    
    # This is the space between the end of the bar and the label
    padding = 4
    
    # Iterate over angles, values, and labels, to add all of them.
    for angle, value, label, in zip(angles, values, labels):
        angle = angle
        
        # Obtain text rotation and alignment
        rotation, alignment = get_label_rotation(angle, offset)

        # And finally add the text
        ax.text(
            x=angle, 
            y=value+padding, 
            s=label, 
            ha=alignment, 
            va="center", 
            rotation=rotation, 
            rotation_mode="anchor",
            fontsize=18,
            fontweight='bold',
            color='white',
            family='Comic Sans MS'
        ) 

In [194]:
df = pd.read_csv('atiba.csv')
group_l = df['group'].unique().tolist()

In [ ]:
# All this part is like the code above
VALUES = df["value"].values
LABELS = df["name"].values
GROUP = df["group"].values

PAD = 6
ANGLES_N = len(VALUES) + PAD * len(np.unique(GROUP))
ANGLES = np.linspace(0, 2 * np.pi, num=ANGLES_N, endpoint=False)
WIDTH = (2 * np.pi) / len(ANGLES)

offset = 0
IDXS = []
GROUPS_SIZE = [36, 48, 40, 46, 42, 46, 40, 42, 44, 36]
for size in GROUPS_SIZE:
    IDXS += list(range(offset + PAD, offset + size + PAD))
    offset += size + PAD

fig, ax = plt.subplots(figsize=(50, 40), subplot_kw={"projection": "polar"})
ax.set_theta_offset(OFFSET)
ax.set_ylim(-100, 100)
ax.set_frame_on(False)
ax.xaxis.grid(False)
ax.yaxis.grid(False)
ax.set_xticks([])
ax.set_yticks([])
fig.patch.set_facecolor('#35363b')
ax.set_facecolor(color='#35363b')

GROUPS_SIZE = [36, 48, 40, 46, 42, 46, 40, 42, 44, 36]
#COLORS = [f"C{i}" for i, size in enumerate(GROUPS_SIZE) for _ in range(size)]

# Colors
COLORS = ["#6C5B7B","#C06C84","#F8B195","#F67280"]

# Colormap
cmap = mpl.colors.LinearSegmentedColormap.from_list("my color", COLORS, N=256)

# Normalizer
norm = mpl.colors.Normalize(vmin=VALUES.min(), vmax=VALUES.max())

# Normalized colors. Each number of tracks is mapped to a color in the 
# color scale 'cmap'
COLORS = cmap(norm(VALUES))

ax.bar(
    ANGLES[IDXS], VALUES, width=WIDTH, color=COLORS, 
    edgecolor="white", linewidth=0.5
)

VALUES2 = df["value_2"].values

add_labels(ANGLES[IDXS], VALUES2, LABELS, OFFSET, ax)

# Extra customization below here --------------------

# This iterates over the sizes of the groups adding reference
# lines and annotations.

offset = 0 
for group, size in zip(group_l, GROUPS_SIZE):
    # Add line below bars
    x1 = np.linspace(ANGLES[offset + PAD], ANGLES[offset + size + PAD - 1], num=50)
    ax.plot(x1, [-5] * 50, color="white")
    
    # Add text to indicate group
    ax.text(
        np.mean(x1), -20, group, color="white", fontsize=40, 
        fontweight="bold", ha="center", va="center",family='Comic Sans MS'
    )
    
    # Add reference lines at 20, 40, 60, and 80
    #x2 = np.linspace(ANGLES[offset], ANGLES[offset + PAD - 1], num=50)
    #ax.plot(x2, [20] * 50, color="#bebebe", lw=0.8)
    #ax.plot(x2, [40] * 50, color="#bebebe", lw=0.8)
    #ax.plot(x2, [60] * 50, color="#bebebe", lw=0.8)
    #ax.plot(x2, [80] * 50, color="#bebebe", lw=0.8)
    #ax.plot(x2, [100] * 50, color="#bebebe", lw=0.8)
    
    offset += size + PAD

cbaxes = inset_axes(
    ax, 
    width="40%", 
    height="40%", 
    loc="center",
    bbox_to_anchor=(0.55, 0.1, 0.35, 0.01),
    bbox_transform=fig.transFigure # Note it uses the figure.
) 


# Create the colorbar
cb = fig.colorbar(
    ScalarMappable(norm=norm, cmap=cmap), 
    cax=cbaxes, # Use the inset_axes created above
    orientation = "horizontal"
)

# Remove the outline of the colorbar
cb.outline.set_visible(False)

# Remove tick marks
cb.ax.xaxis.set_tick_params(labelsize=20,colors="white")

ax.set_title('Atiba Hutchinson / Minutes Played per Match 2013-2023', y=1.08, pad=-14,size=40,color='white',fontweight='bold',family='Comic Sans MS')
#ax.set_title('@seripenaltilar', y=0, pad=-14,size=40,color='#676c70',fontweight='bold',family='Comic Sans MS')

plt.figtext(0.60,0.2,"@seripenaltilar",fontsize=25,fontweight='bold',color='#676c70')

plt.show()

In [201]:
ANGLES

array([0.        , 0.01308997, 0.02617994, 0.03926991, 0.05235988,
       0.06544985, 0.07853982, 0.09162979, 0.10471976, 0.11780972,
       0.13089969, 0.14398966, 0.15707963, 0.1701696 , 0.18325957,
       0.19634954, 0.20943951, 0.22252948, 0.23561945, 0.24870942,
       0.26179939, 0.27488936, 0.28797933, 0.3010693 , 0.31415927,
       0.32724923, 0.3403392 , 0.35342917, 0.36651914, 0.37960911,
       0.39269908, 0.40578905, 0.41887902, 0.43196899, 0.44505896,
       0.45814893, 0.4712389 , 0.48432887, 0.49741884, 0.51050881,
       0.52359878, 0.53668874, 0.54977871, 0.56286868, 0.57595865,
       0.58904862, 0.60213859, 0.61522856, 0.62831853, 0.6414085 ,
       0.65449847, 0.66758844, 0.68067841, 0.69376838, 0.70685835,
       0.71994832, 0.73303829, 0.74612826, 0.75921822, 0.77230819,
       0.78539816, 0.79848813, 0.8115781 , 0.82466807, 0.83775804,
       0.85084801, 0.86393798, 0.87702795, 0.89011792, 0.90320789,
       0.91629786, 0.92938783, 0.9424778 , 0.95556777, 0.96865

480